In [11]:
from edgar import *
from edgar import set_identity

set_identity("Neal Lockhart nhl3388@uncw.edu")
filings = Company("NOW").get_filings(form="10-K").latest(2)
filings[0]


[16:08:41] INFO     Identity of the Edgar REST client set to [Neal Lockhart nhl3388@uncw.edu]           ]8;id=434084;file://c:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\edgar\core.py\core.py]8;;\:]8;id=597490;file://c:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\edgar\core.py#161\161]8;;\

╭────────────────────────────────────── ServiceNow, Inc. [1373715] 10-K 📊 ───────────────────────────────────────╮
│ ╭──────────────────────┬────────────╮                                                                           │
│ │ 0001373715-24-000030 │ 2024-01-25 │                                                                           │
│ ╰──────────────────────┴────────────╯                                                                           │
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮                  │
│ │ Links: 🏠 Homepage 📄 Primary Document 📜 Full Submission Text                             │                  │
│ ├────────────────────────────────────────────────────────────────────────────────────────────┤                  │
│ │ 🏠 https://sec.gov/Archives/edgar/data/1373715/0001373715-24-000030-index.html             │                  │
│ │ 📄 https://sec.gov/Archives/edgar/data/1373715/000137371524000030/now-2023

In [12]:
md = filings[0].sections()

In [13]:
#search for the word "risk" in the 10-k sections list
a_list=[]
for i in range(len(md)):
    if "item 1a." in md[i].lower():
        index_start=i
    if "item 1b." in md[i].lower():
        index_end=i

md[index_start:index_end]

['ITEM 1A.       RISK FACTORS',
 'Investing in our securities involves risks. You should carefully consider the risks and uncertainties described below, together with the other information in this Annual Report on Form 10-K, before making an investment decision. The occurrence of any of the following risks, or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial, could materially and adversely affect our business, financial condition or results of operations or cause our stock price to decline. The following risks have been grouped by categories and are not in order of significance or probability of occurrence.',
 'Risk Factors Summary\nThis summary provides an overview of the risks we face and should not be considered a substitute for the more fulsome risk factors discussed immediately following this summary. \n•Risks Related to Our Ability to Grow Our Business\n•Laws, regulations and customer expectations regarding the use, storag

In [14]:
#turn list into string
md_str = ' '.join(md[index_start:index_end])

In [15]:
md_str

'ITEM 1A.       RISK FACTORS Investing in our securities involves risks. You should carefully consider the risks and uncertainties described below, together with the other information in this Annual Report on Form 10-K, before making an investment decision. The occurrence of any of the following risks, or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial, could materially and adversely affect our business, financial condition or results of operations or cause our stock price to decline. The following risks have been grouped by categories and are not in order of significance or probability of occurrence. Risk Factors Summary\nThis summary provides an overview of the risks we face and should not be considered a substitute for the more fulsome risk factors discussed immediately following this summary. \n•Risks Related to Our Ability to Grow Our Business\n•Laws, regulations and customer expectations regarding the use, storage and mov

In [23]:
#make dataframe with md_stry as a column and ticker NOW as another column
import pandas as pd
df = pd.DataFrame(['NOW'], columns=['ticker'])
df['text'] = md_str
df

,ticker,text
0,NOW,ITEM 1A. RISK FACTORS Investing in our s...


In [34]:
#make into csv
df.to_csv('NOW_10k.csv', index=False, sep='|',header=False)

In [28]:
#save string to csv
import csv
with open('ServiceNow_10k.csv', 'w') as f:
    f.write(md_str)

In [47]:
from langchain_community.document_loaders.csv_loader import CSVLoader
file_path = ('NOW_10k.csv')
loader = CSVLoader(file_path=file_path,
                   csv_args={
                    "delimiter":"|",
                    "quotechar":'"',
                    "fieldnames":["ticker", "text"]
                   },
                   encoding="utf-8"
)

data = loader.load()

In [36]:
data

[Document(metadata={'source': 'NOW_10k.csv', 'row': 0}, page_content='ticker: NOW\ntext: ITEM 1A.       RISK FACTORS Investing in our securities involves risks. You should carefully consider the risks and uncertainties described below, together with the other information in this Annual Report on Form 10-K, before making an investment decision. The occurrence of any of the following risks, or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial, could materially and adversely affect our business, financial condition or results of operations or cause our stock price to decline. The following risks have been grouped by categories and are not in order of significance or probability of occurrence. Risk Factors Summary\nThis summary provides an overview of the risks we face and should not be considered a substitute for the more fulsome risk factors discussed immediately following this summary. \n•Risks Related to Our Ability to Grow Our B

In [48]:
from langchain_community.document_loaders.csv_loader import CSVLoader
file_path = ('NOW_10k.csv')
loader = CSVLoader(file_path=file_path,
                   csv_args={
                    "delimiter":"|",
                    "quotechar":'"',
                    "fieldnames":["ticker", "text"]
                   },
                   encoding="utf-8")
data = loader.load()


from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
texts=text_splitter.split_documents(data)

texts[0:20]

[Document(metadata={'source': 'NOW_10k.csv', 'row': 0}, page_content='ticker: NOW'),
 Document(metadata={'source': 'NOW_10k.csv', 'row': 0}, page_content='text: ITEM 1A.       RISK FACTORS Investing in our securities involves risks. You should carefully consider the risks and uncertainties described below, together with the other information in this Annual Report on Form 10-K, before making an investment decision. The occurrence of any of the following risks, or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial, could materially and adversely affect our business, financial condition or'),
 Document(metadata={'source': 'NOW_10k.csv', 'row': 0}, page_content='to be immaterial, could materially and adversely affect our business, financial condition or results of operations or cause our stock price to decline. The following risks have been grouped by categories and are not in order of significance or probability of occurrence. Risk F

In [49]:
import os

os.environ["AZURE_OPENAI_API_KEY"]="b229fdc63a3d4b06b4adcc67660474f3"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://openai-541.openai.azure.com/"

In [50]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint='https://aiall9596864698.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15',
    api_key='6d440529fad24ffc8aef6d5f9ef52593',
    openai_api_version="2024-07-01-preview"
)

In [69]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="my_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db2"
)

In [52]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(texts))]
vector_store.add_documents(documents=texts, ids=uuids)

['15a2d9dc-e8b3-49ec-a192-0586b4aab509',
 'd798924e-44bb-48f3-a467-4824911423fc',
 'b13b24bb-00e5-4004-96b0-13c2f92c1538',
 '52ce9fc1-4998-44e2-b66f-dba8d375c245',
 '34bbb077-3ddb-4974-b910-81d792469739',
 '3feb51b3-8a0d-4c29-8d54-a8d08d45a406',
 '47bece71-00b6-468a-8597-1cb86d8c4843',
 '41fa83e0-60ca-4c92-99b4-884bc873a9de',
 '78384ca8-9d81-4494-ac00-bf5901165108',
 'c306bf56-d1d2-466f-9a07-d7fafcfc8dd0',
 'a6be6c3c-6cac-4013-9ece-652acd3b2cf2',
 '41e7a2ec-6512-4e9c-816d-b2c5375afc67',
 '8a446f5c-d43a-42e3-8ebb-580ea87ddd3a',
 '225b3f81-b19b-4550-824c-d0ebb08434ba',
 '970891f8-516c-4414-ad8f-4920c21af73c',
 '51eb0cd2-30e9-407e-9daa-82d82e40214b',
 '891966f6-b5f7-4c2c-b4fd-1f9a55d51d51',
 '8cbfec54-75b4-40ee-aa1f-21d5205e247b',
 '1c8d2e4b-6bdd-495e-9116-dc8e9ec16fdb',
 '59a92fad-33d1-49d1-9228-6fce19d9cfd1',
 'a07f2482-d900-4805-8c6f-1b4ca3bf5496',
 '08e075b9-dd91-4d2a-9407-72f7daac91ac',
 '7adb4c1f-61c8-484b-9970-d381c8e33f13',
 '2af2b8e1-3bb9-4bee-ae47-cb2340656a52',
 'db8b7049-d3ff-

In [53]:
retriever = vector_store.as_retriever()

In [70]:
docs=retriever.invoke('How is litigation play a role in risk?')

In [76]:
print(docs[1].page_content)

accruals, other increased expenses or risks of litigation. We may not have insurance sufficient to compensate us for potentially significant losses that may result from claims arising from disruptions to our services.


In [56]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  # or your deployment
    api_version="2024-07-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [66]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.  The context is from the item 1a in a 10-k sec filing for the ticker NOW.  Always quote the context in your answer.
{question}
Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [67]:
response = rag_chain.invoke("How is litiagation play a role in risk?")

print(response.content)

Litigation plays a role in risk as securities class action litigation could result in substantial costs and divert management's attention and resources from the business, which could materially adversely affect the business, operating results, and financial condition. Additionally, any claim or litigation against the company could be costly, time-consuming, and divert the attention of management and key personnel.
